Resolveremos la ecuación de Stokes 2D para un lid-driven cavity flow, que nos establece ciertas condiciones de borde. 

$$-\mu \nabla^2 u + \nabla p = f$$
$$ \nabla \cdot u = 0$$

Condiciones de borde (Dirichlet)
$$ u = (1,0) \quad \text{en} \ \partial \Omega_0$$ 
$$ u = (0,0) \quad \text{en} \ \partial \Omega_1$$ 

Con $\partial \Omega_1 = ([0, 1]\times 0) \cup (0 \times [0, 1]) \cup (1 \times [0,1]) $

Y
$\partial \Omega_0 = [0,1]\times 1$

# **Implementación**

## *Librerías*

Lo primero que hacemos es importar "MPI". [MPI](https://en.wikipedia.org/wiki/Message_Passing_Interface) o Message Passing Interface es un estándar que define la sintaxis y semánticas de distintas librerias. Está diseñada para funcionar en arquitecturas de computación paralela

In [1]:
from mpi4py import MPI # Message Passing Interface.
# Is an standar for different libraries to function properly and with high performance

Luego, importamos PETSc y dolfinx. [PETSc](https://petsc.org/release/) es una librería potente usada para computación científica, en particular, resolver problemas lineares grandes asociados a ecuaciones diferenciales parciales. Por otro lado, [dolfinx](https://fenicsproject.org/) es la interfaz que usa FeniCSx, una librería de código abierto especializada para la resolución de ecuaciones diferenciales parciales bajo el método de elementos finitos (FEM).

In [2]:
from petsc4py import PETSc # Library for solving PDE's assosiated linear problems
import dolfinx # FeniCSx Interface

In [3]:
import numpy as np # Numpy, the classic

Ahora en la siguiente matraca, se importan escencialmente todas las funciones que estaremos utilizando para plantear el problema de stokes

In [4]:
import numpy as np # Numpy, the classic

import ufl
from basix.ufl import element, mixed_element # Elements for the FEM
from dolfinx import default_real_type, fem, la # FEM library with the functions required for solving through FEM (for real values)
from dolfinx.fem import (
    Constant,
    Function,
    dirichletbc,
    extract_function_spaces,
    form,
    functionspace,
    locate_dofs_topological,
    create_interpolation_data,
    assemble_scalar,
    Expression
) # Functions for defining variational problems and defining mathematical functions and constants
from dolfinx.fem.petsc import assemble_matrix, assemble_vector, apply_lifting # Functions for creating the matrix problems
from dolfinx.io import XDMFFile
from dolfinx.mesh import CellType, create_rectangle, locate_entities_boundary # Mesh creating functions
from ufl import div, dx, grad, inner, curl # operand functions

## *Dominio y Malla*

In [5]:
# Malla como el cuadrado unitario
n = 128
msh = create_rectangle(MPI.COMM_WORLD, [np.array([0,0]), np.array([1,1])],  [n,n],  CellType.triangle)
# Se especifica MPI.COMM_WORLD para paralelizar el proceso de crear la malla

# El primer array refiere a la lista de puntos que definen el rectángulo (2 puntos)

# El 2do array refiere a la "resolución" de la malla. [n_x,n_y]
# n_x es el número de segmentos en que se divide el dominio en el eje x
# Análogo con n_y. De esta forma h = 1/n_x

# El número de celdas sería n_x*n_y, y el tipo especificado por "CellType.type", en este caso triangulares


Luego de crear la malla, identificamos los bordes del dominio para después asignarles las condiciones a cada uno

In [6]:
# Ahora especificamos aquellas partes del dominio que corresponden al borde
# Esto por medio de funciones asociadas a estas zonas

# Para Omega_1
def noslip_boundary(x):
    # Dado un punto x, retorna si su coordenada x[i] es cercana al valor dado después.
    # En este caso, muro izquierdo | muro derecho | muro inferior
    return np.isclose(x[0], 0.0) | np.isclose(x[0], 1.0) | np.isclose(x[1], 0.0)

# Para Omega_0
def lid(x):
    # Muro superior
    return np.isclose(x[1], 1.0)

Especificamos una condición de borde extra para las esquinas

In [7]:
def corners(x):
    # detecta las esquinas. 
    return (np.isclose(x[0], 0.0) & np.isclose(x[1], 1.0)) | \
           (np.isclose(x[0], 1.0) & np.isclose(x[1], 1.0))

## *Elementos y espacios de funciones*

Ahora, podemos definir los espacios de funciones que queremos utilizar para la velocidad y la presión.

In [8]:
# Utilizaremos Taylor-Hood elements para la malla

# P2 para la velocidad, "Lagrange" es el nombre. Tiene un grado k = 2
P2 = element("Lagrange", msh.basix_cell(), 2, shape=(msh.geometry.dim,), dtype=default_real_type)
# msh.basix_cell() es para saber la geometría de nuestros elementos, en este caso triángulos
# shape=1 es default, refiere a la dimensión que tendrá el elemento, en este caso 2, pues es un vector con 2 coordenadas
#dtype es el tipo de número que se utilizará, en este caso Reales

# P1 para la presión, "Lagrange" es el nombre. Tiene un grado k-1 = 1
P1 = element("Lagrange", msh.basix_cell(), 1, dtype=default_real_type)

# Creamos el espacio mixto
TH = mixed_element([P2, P1])
W = functionspace(msh, TH)

## *Condiciones de borde*

In [9]:
W0 = W.sub(0) # Tomamos la parte del espacio mixto que corresponde a la velocidad
Q, _ = W0.collapse() # Creamos este "nuevo" espacio de funciones correspondiente a la velocidad

# no-slip boundary conditions
noslip = Function(Q) # Crea una función en el espacio Q con default 0 (u = (0,0))
#u0 = Constant(msh, (PETSc.ScalarType(0.0), PETSc.ScalarType(0.0)))
#noslip.interpolate(u0)


facets = locate_entities_boundary(msh, 1, noslip_boundary) # (msh, dim, marker)
# Esta función identifica en msh los puntos de dim=1 dados por la función noslip_boundary

# Asignamos la condición de Dirichlet u = (0,0) a esos puntos hallados
dofs = locate_dofs_topological((W0, Q), 1, facets)
bc0 = dirichletbc(noslip, dofs, W0)
# Plantearlo con (W0,Q) permite que se mapee correctamente de vuelta a W0 "The tuple (W0, Q) lets DOLFINx map back from Q to W0 correctly"

In [10]:
# Función para definir la velocidad del flujo en Omega_0, el muro superior
def lid_velocity_expression(x):
    # Retorna la velocidad u = (1,0)
    return np.stack((np.ones(x.shape[1]), np.zeros(x.shape[1])))

In [11]:
# Lid-driven boundary condition u = (1,0) en y=1
lid_velocity = Function(Q)
lid_velocity.interpolate(lid_velocity_expression) # Interpola valores dados por lid_velocity_expression con funciones del espacio de funciones de Q
facets = locate_entities_boundary(msh, 1, lid)

dofs = locate_dofs_topological((W0, Q), 1, facets)
bc1 = dirichletbc(lid_velocity, dofs, W0)


In [12]:
# Lid-driven boundary condition u = (1,0) en y=1
#corner_bc = Function(Q)
#corner_bc.interpolate(lid_velocity_expression) # Interpola valores dados por lid_velocity_expression con funciones del espacio de funciones de Q
#facets = locate_entities_boundary(msh, 1, corners)

#dofs = locate_dofs_topological((W0, Q), 1, facets)
#c2 = dirichletbc(corner_bc, dofs, W0)


# Juntamos en una lista las condiciones de borde
bcs = [bc0,bc1]

## *Formulación del problema*

Primero creamos las test & trial functions, junto con $f$

In [13]:
# Definimos el problema variacional
(u,p) = ufl.TrialFunctions(W) # Definimos las trial functions
(v,q) = ufl.TestFunctions(W) # Definimos las test functions

f = Function(Q) 
#f0 = Constant(mesh, (PETSc.ScalarType(0), PETSc.ScalarType(0)))
#f.interpolate(f0)  

Ahora, procedemos a plantear las formas lineal y bilineal

Forma variacional

$$
\mu \int_\Omega \nabla u : \nabla v \ d\Omega - \int_\Omega p \nabla \cdot v \ d\Omega = \int_\Omega f \cdot v \ d\Omega
$$
$$ 
\int_\Omega (\nabla \cdot u) \cdot q \ d\Omega
$$

In [14]:
U = 1 # Characteristic velocity
L = 1 # Characteristic lenght
mu = 1e-8
rho = 1e-4

nu = mu/rho
Re = (U*L)/nu
k = 1/Re
print(Re)

10000.0


In [15]:
# Forma bilineal
#a = form(PETSc.ScalarType(mu)*(inner(grad(u), grad(v)) - inner(p, div(v)) + inner(div(u), q))* dx )
a = form(PETSc.ScalarType(k)*(inner(grad(u), grad(v)) - inner(p, div(v)) + inner(div(u), q))* dx )

# Forma lineal
L = form(inner(f,v) * dx)

Armamos la matriz y el vector asociados al problema

In [16]:
# Armamos la matriz A de la forma bilinear
A = assemble_matrix(a, bcs=bcs)
A.assemble()

# Armamos el vector b de la forma linear
b = assemble_vector(L)

# "Ajustamos" b para que se mantenga la igualad Ax = b luego de aplicarle las bcs a A
apply_lifting(b, [a], bcs=[bcs])

# Sincroniza los "Ghost values" usados para la computación paralela
b.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)

# Aplicamos las condiciones de borde a b
for bc in bcs:
    bc.set(b)

## _**Resolución del problema**_

Solver

In [17]:
# Configuramos el solver (Kyrlov SubsPace method)
ksp = PETSc.KSP().create(msh.comm)
# KSP refiere a usar krylov subspaces para resolver el problema
# create(msh.comm) hace que el solver utilice ksp

ksp.setOperators(A) # Define la matriz A del sistema
ksp.setType("preonly") # Aplica el precondicionador 1 sola vez de forma que sea un direct solver
#ksp.setType("cg") # Utiliza el método de gradiente conjugado para resolver el sistema lineal

Nullspace

In [18]:
# Configuramos MUMPS para que maneje el espacio nulo de la presión
# MUMPS: MUltifrontal Massively Parallel sparse direct Solver
pc = ksp.getPC() # Precondicionador de ksp
pc.setType("lu") # El preocondicionador a usar (de LU)

pc.setFactorSolverType("mumps") 
pc.setFactorSetUpSolverType()
pc.getFactorMatrix().setMumpsIcntl(icntl=24, ival=1)
pc.getFactorMatrix().setMumpsIcntl(icntl=25, ival=0)

pc.setFactorSolverType("superlu_dist")

Resolución

In [19]:
U = Function(W)
ksp.solve(b, U.x.petsc_vec)

In [20]:
u, p = U.sub(0).collapse(), U.sub(1).collapse()

In [21]:
type(u)

dolfinx.fem.function.Function

In [22]:
V_curl = functionspace(msh, ("CG", 1)) # Discontinuous Lagrange space de grado k-1
vortex = curl(u)
vorticity_exp = Expression(vortex, V_curl.element.interpolation_points())

# Interpolamos la función (espacio) sobre los valores de curl(u) 
vorticity = Function(V_curl)
vorticity.interpolate(vorticity_exp)

In [23]:
from dolfinx.io import XDMFFile

# Guardar el archivo de la velocidad
with XDMFFile(MPI.COMM_WORLD, "xdmf/velocity.xdmf", "w") as ufile_xdmf:
    u.x.scatter_forward()
    P1 = element(
        "Lagrange", msh.basix_cell(), 1, shape=(msh.geometry.dim,), dtype=default_real_type
    )
    u1 = Function(functionspace(msh, P1))
    u1.interpolate(u)
    ufile_xdmf.write_mesh(msh)
    ufile_xdmf.write_function(u1)

# Guardar la presión
with XDMFFile(MPI.COMM_WORLD, "xdmf/pressure.xdmf", "w") as pfile_xdmf:
    p.x.scatter_forward()
    pfile_xdmf.write_mesh(msh)
    pfile_xdmf.write_function(p)

In [24]:
print(len(u.x.array))
print(len(u1.x.array))

132098
33282


In [25]:
print(len(u.x.array))
print(len(p.x.array))

132098
16641


## _**Vorticidad**_

In [26]:
with XDMFFile(msh.comm, "xdmf/vorticity.xdmf", "w") as vortfile_xdmf:
    vorticity.x.scatter_forward()
    vortfile_xdmf.write_mesh(msh)
    vortfile_xdmf.write_function(vorticity)